In [1]:
import sys
sys.path.append('..')

from scripts.autoencoders import InMemoryImageDataset, split_images, ConfigurableAutoencoder
from scripts.measuring_quality import first_order_method, co_ocurrence_matrix, deltah, second_order_method

import torch
from torchvision import transforms
from torch.utils.data import DataLoader
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from omegaconf import OmegaConf
import warnings
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image

In [2]:
try:
    OmegaConf.register_new_resolver("eval", eval)
except ValueError:
    pass

Elegir el archivo de configuración correspondiente:

In [3]:
config_name = 'config_1' # Elegir

config_path = f'configs/imagenes_reales/{config_name}.yaml'
config = OmegaConf.load(config_path)
config

{'training': {'num_epochs': 300, 'batch_size': 64, 'learning_rate': 0.001, 'scheduler_name': 'elr', 'scheduler_params': {'gamma': 0.95}, 'side_size': 512}, 'model': {'encoding_dim': 1600, 'loss_function': 'mse', 'optimizer': 'adam'}, 'encoder': {'layers': [{'type': 'conv2d', 'filters': 8, 'kernel_size': 8, 'stride': 4, 'padding': 2, 'activation': 'relu'}, {'type': 'conv2d', 'filters': 16, 'kernel_size': 8, 'stride': 4, 'padding': 2, 'activation': 'relu'}, {'type': 'flatten'}, {'type': 'dense', 'dim': '${model.encoding_dim}', 'activation': 'relu'}]}, 'decoder': {'layers': [{'type': 'dense', 'dim': 16384, 'activation': 'relu'}, {'type': 'unflatten', 'dim1': 32, 'dim2': 32, 'out_channels': 16}, {'type': 'conv2d_transpose', 'filters': 8, 'kernel_size': 8, 'stride': 4, 'padding': 2, 'activation': 'relu'}, {'type': 'conv2d_transpose', 'filters': 1, 'kernel_size': 8, 'stride': 4, 'padding': 2, 'activation': 'sigmoid'}]}, 'testing': {'batch_size': 32}}

Cargo el autoencoder ya entrenado:

In [4]:
image_size = config['training']['side_size']

In [5]:
# 1. Crear una instancia del modelo (debe tener la misma arquitectura)
autoencoder_cargado = ConfigurableAutoencoder(config=config, image_size=image_size)
# 2. Carga los parámetros
autoencoder_cargado.load_state_dict(torch.load(f'data/trained_models/imagenes_reales/{config_name}.pth'))
# 3. Modo evaluación (cuando lo use para inferencia)
autoencoder_cargado.eval()

ConfigurableAutoencoder(
  (encoder): Sequential(
    (0): Conv2d(1, 8, kernel_size=(8, 8), stride=(4, 4), padding=(2, 2))
    (1): ReLU()
    (2): Conv2d(8, 16, kernel_size=(8, 8), stride=(4, 4), padding=(2, 2))
    (3): ReLU()
    (4): Flatten(start_dim=1, end_dim=-1)
    (5): Linear(in_features=16384, out_features=1600, bias=True)
    (6): ReLU()
  )
  (decoder): Sequential(
    (0): Linear(in_features=1600, out_features=16384, bias=True)
    (1): ReLU()
    (2): Unflatten(dim=1, unflattened_size=(16, 32, 32))
    (3): ConvTranspose2d(16, 8, kernel_size=(8, 8), stride=(4, 4), padding=(2, 2))
    (4): ReLU()
    (5): ConvTranspose2d(8, 1, kernel_size=(8, 8), stride=(4, 4), padding=(2, 2))
    (6): Sigmoid()
  )
)

Genero dataset de testeo:

In [6]:
def load_tiff_datasets(carpeta):
    carpeta_archivos = f"C:/Users/Bianca/Documents/Archivos/ITBA/Tesis_Maestria/SAR_despeckling_filters_dataset/Main_folder/{carpeta}/"
    archivos_tiff = [f for f in os.listdir(carpeta_archivos) if f.endswith('.tiff')]

    num_imagenes = len(archivos_tiff)
    array_final = np.empty((num_imagenes, 512, 512), dtype=np.uint8)

    for i, archivo in enumerate(archivos_tiff):
        imagen = Image.open(os.path.join(carpeta_archivos, archivo))
        array_imagen = np.array(imagen)[:, :, 0]
        array_final[i] = array_imagen
        
        if (i+1) % 250 == 0:
            print(f'{i+1} de {num_imagenes} imágenes procesadas')

    print(f'\nArray final shape: {array_final.shape}')
    print(f'Máximo valor: {array_final.max()}, mínimo valor: {array_final.min()}')
    
    return array_final

In [7]:
noisy_val = load_tiff_datasets('Noisy_val')


Array final shape: (100, 512, 512)
Máximo valor: 255, mínimo valor: 0


In [8]:
clean_val = load_tiff_datasets('GTruth_val')


Array final shape: (100, 512, 512)
Máximo valor: 255, mínimo valor: 0


In [9]:
clean_val = clean_val[np.array([np.min(img) != np.max(img) for img in clean_val])]
noisy_val = noisy_val[np.array([np.min(img) != np.max(img) for img in noisy_val])]
print(clean_val.shape)
print(noisy_val.shape)

(100, 512, 512)
(100, 512, 512)


In [12]:
autoencoder_cargado.eval() # Para desactivar Dropout, BatchNorm, etc.
batch_size = config['testing']['batch_size']

In [13]:
if config['training']['side_size'] == 512:
    pass

else:
    clean_val = split_images(clean_val, config['training']['side_size'])
    noisy_val = split_images(noisy_val, config['training']['side_size'])    
    
    clean_val = clean_val[np.array([np.min(img) != np.max(img) for img in clean_val])]
    noisy_val = noisy_val[np.array([np.min(img) != np.max(img) for img in noisy_val])]
    
print(clean_val.shape)
print(noisy_val.shape)

(100, 512, 512)
(100, 512, 512)


In [15]:
normalize_to_01 = transforms.Lambda(lambda x: (x - x.min()) / (x.max() - x.min()))

transform = transforms.Compose([
    transforms.ToTensor(),
    normalize_to_01
])

dataset_test = InMemoryImageDataset(noisy_val, clean_val, transform=transform)
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)